<a href="https://colab.research.google.com/github/kkorhone/Python_Notebooks/blob/main/jtf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
pip install pygfunction

In [21]:
import matplotlib.pyplot as plt
import scipy.interpolate
import scipy.optimize
import scipy.signal
import pygfunction
import numpy as np

In [43]:
def calculate_thermal_resistance(borehole, r_inner, r_outer, shank_spacing, k_rock, verbal=False, visual=False):

    if verbal:
      print(f"calculate_thermal_resistance(borehole={borehole}, r_inner={r_inner}, r_outer={r_outer}, shank_spacing={shank_spacing}, k_rock={k_rock})")

    epsilon = 1.0e-6      # Pipe roughness [m]
    k_pipe = 0.42         # Pipe thermal conductivity [W/(m*K)]
    k_grout = 0.60        # Grout thermal conductivity [W/(m*K)]
    flow_rate = 50 / 60   # Fluid flow rate [kg/s] = [L/s]
    Cp_fluid = 3795       # Fluid specific isobaric heat capacity [J/(kg*K)]
    rho_fluid = 1052      # Fluid density (kg/m3)
    mu_fluid = 0.0052     # Fluid dynamic viscosity [Pa*s]
    k_fluid = 0.48        # Fluid thermal conductivity [W/(m*K)]

    positions = [(-0.5*shank_spacing, 0), (0.5*shank_spacing, 0)]

    if verbal:
        print(f"positions={positions}")

    R_pipe = pygfunction.pipes.conduction_thermal_resistance_circular_pipe(r_inner, r_outer, k_pipe)

    if verbal:
        print(f"R_pipe={R_pipe}")

    h_fluid = pygfunction.pipes.convective_heat_transfer_coefficient_circular_pipe(flow_rate, r_inner, mu_fluid, rho_fluid, k_fluid, Cp_fluid, epsilon)
    R_fluid = 1 / (h_fluid * 2 * np.pi * r_inner)

    if verbal:
        print(f"h_fluid={h_fluid}")
        print(f"R_fluid={R_fluid}")

    u_tube = pygfunction.pipes.SingleUTube(positions, r_inner, r_outer, borehole, k_rock, k_grout, R_fluid+R_pipe)

    R_borehole = pygfunction.pipes.borehole_thermal_resistance(u_tube, flow_rate, Cp_fluid)

    if verbal:
        print(f"R_borehole={R_borehole:.3f}")

    check = u_tube._check_geometry()

    if not check:
        raise ValueError("Invalid borehole geometry")

    if visual:
        u_tube.visualize_pipes()

    return R_borehole

In [44]:
def calculate_annual_energy(monthly_fraction, L_borehole, verbal=False, visual=False):

    if verbal:
        print(f"calculate_annual_energy(monthly_fraction={monthly_fraction}, L_borehole={L_borehole})")

    assert np.abs(np.sum(monthly_fraction) - 1) < 1e-6

    T_surface = 6.1                         # [degC]
    q_geothermal = 0.053                    # [W/m^2]

    k_rock = 3.84                           # [W/(m*K)]
    Cp_rock = 728.0                         # [J/(kg*K)]
    rho_rock = 2619.0                       # [kg/m^3]

    r_borehole = 0.140 / 2                  # [m]

    num_years = 25                          # [1]

    T_target = -1.0                         # [degC]

    a_rock = k_rock / (rho_rock * Cp_rock)  # [m^2/s]

    t_max = num_years * 365 * 24 * 3600.0   # [s]

    delta_t = 730 * 3600.0                  # [s]

    r_outer = 0.055 / 2                     # [m]
    r_inner = r_outer - 0.0033              # [m]

    shank_spacing = 0.075

    T_initial = T_surface + q_geothermal / k_rock * (0.5 * L_borehole)

    if verbal:
        print(f"C_rock={rho_rock*Cp_rock/1e6:.1f}")
        print(f"T_initial={T_initial:.1f}")

    borehole = pygfunction.boreholes.Borehole(H=L_borehole, D=0, r_b=r_borehole, x=0, y=0)

    R_borehole = calculate_thermal_resistance(borehole, r_inner, r_outer, shank_spacing, k_rock, verbal=verbal, visual=visual)

    t = pygfunction.utilities.time_geometric(delta_t, t_max, 50)
    g = pygfunction.gfunction.uniform_temperature([borehole], t, a_rock, nSegments=32, disp=False)

    ti = np.arange(delta_t, t_max+delta_t, delta_t)
    gi = scipy.interpolate.interp1d(t, g)(ti)

    def evaluate_mean_fluid_temperatures(E_annual):
        monthly_heat_load = E_annual * monthly_fraction
        heat_rate = np.ravel(np.tile(monthly_heat_load*1_000_000/730.0, (1, num_years)))
        specific_heat_rate = heat_rate / L_borehole
        delta_q = np.hstack((-specific_heat_rate[0], np.diff(-specific_heat_rate)))
        T_wall = T_initial + scipy.signal.fftconvolve(delta_q, gi/(2.0*np.pi*k_rock), mode="full")[:len(ti)]
        T_fluid = T_wall - R_borehole * specific_heat_rate
        return T_fluid

    def cost_function(E_annual):
        T_fluid = evaluate_mean_fluid_temperatures(E_annual)
        return np.abs(np.min(T_fluid) - T_target)

    E_annual = scipy.optimize.fminbound(cost_function, 1, 1000, xtol=0.001)

    if verbal:
        print(f"E_annual={E_annual:.3f}")

    if visual:
        T_fluid = evaluate_mean_fluid_temperatures(E_annual)
        plt.figure()
        plt.plot(ti/(365*24*3600), T_fluid)
        plt.axhline(T_target, ls="--", color="k")
        plt.xlabel("Year")
        plt.ylabel(u"Mean fluid temperature [\xb0C]")
        plt.title(f"E_annual = {E_annual:.3f} MWh")

    return E_annual

In [57]:
if __name__ == "__main__":

    print("Constant heat extraction profile:")
    print("L_borehole=600", "E_annual="+str(calculate_annual_energy(np.ones(12)/12, 600)))
    print("L_borehole=800", "E_annual="+str(calculate_annual_energy(np.ones(12)/12, 800)))

    print("Helsinki heat extraction profile:")
    print("L_borehole=600", "E_annual="+str(calculate_annual_energy(np.array([0.177,0.111,0.111,0.082,0.044,0.037,0.032,0.034,0.044,0.086,0.119,0.123]), 600)))
    print("L_borehole=800", "E_annual="+str(calculate_annual_energy(np.array([0.177,0.111,0.111,0.082,0.044,0.037,0.032,0.034,0.044,0.086,0.119,0.123]), 800)))

    print("EED heat extraction profile:")
    print("L_borehole=600", "E_annual="+str(calculate_annual_energy(np.array([0.155,0.148,0.125,0.099,0.064,0.000,0.000,0.000,0.061,0.087,0.117,0.144]), 600))) # EED=83.3 pygfunction=85.8 difference=3%
    print("L_borehole=800", "E_annual="+str(calculate_annual_energy(np.array([0.155,0.148,0.125,0.099,0.064,0.000,0.000,0.000,0.061,0.087,0.117,0.144]), 800))) # EED=119.3 pygfunction=121.7 difference=2%


Constant heat extraction profile:
L_borehole=600 E_annual=140.8504430421353
L_borehole=800 E_annual=200.75851230890603
Helsinki heat extraction profile:
L_borehole=600 E_annual=76.99348895023051
L_borehole=800 E_annual=109.10929224315885
EED heat extraction profile:
L_borehole=600 E_annual=85.81072378816361
L_borehole=800 E_annual=121.7324581567558
